<a href="https://colab.research.google.com/github/foopoiuyt/uplimit/blob/main/%5BStudent_Version%5D_Langchain_Week1_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook is part of the course: [LLM Apps with Langchain](https://uplimit.com/course/llm-apps-with-langchain) and is created by Sidharth Ramachandran as the project for Week 1 of the course.

# Let's build "PillPal" - a Telegram bot and website that allows users to ask questions based on the Patient Information Leaflet (PIL) or drug guide that comes with any medication.

![Package Information Leaflet](https://i.ibb.co/ZSdGyjV/ima-image-36904.jpg)

I'm sure you have all come across the thin piece of folded paper that is part of every drug prescription box. Usually the text is in very small print and typically provides information about dosages, side effects, storage instructions and much more. They are hard to read and understand and requires some effort to get answers to common questions a patient might have. What if we could create a product that answers these questions and actually makes the medical information more accessible and easier to understand - enter PillPal!

In the process of building PillPal, we will go through the following stages and on the way learn more about LLM Apps and their typical life-cycle.

1. BUILD the app: this is where we first test the idea, try multiple options for various parts of the pipeline till we are satisfied to a reasonable extent that the product works.
2. DEPLOY & MONITOR the app: this is where we want to make the app available to our first users, monitor it's behaviour and discover edge cases.
3. EVALUATE & IMPROVE the app: this is the final and ongoing stage where we will build an evaluation suite that constantly checks whether our app is working as expected and perform experiments.

We will use Langchain to develop the app, deploy it as a Telegram bot/ Website and use Langsmith to monitor, evaluate and improve the product. Please consider PillPal as the chosen example/ case study to illustrate the process but feel free to adapt this project ot build any app of your choice!

# # 👨‍🎓 Learner Project

In the project walkthrough session we will go through the various steps of the project with an example of one of the drugs. The learner project is to implement the same steps for a different drug OR any other PDF of your choice. Concretely, here are steps that you need to do:

1. Identify a PDF that you would like to ask questions about. Suggestion: choose another drug or medication that you are familiar with.
2. Upload and use that PDF as you go through this notebook.
3. Make use of the sections and flow as a guide but you are expected to update and write code to complete the RAG project for your PDF.
4. Feel free to adapt any sections to add more functionality or adapt for your use case/ PDF.

# Step 0: Necessary libraries and setup

In [4]:
# Install the necessary libraries to get it out of the way
!pip install langchain
!pip install langchain-community
!pip install langchain-openai
!pip install faiss-cpu
!pip install pymupdf
!pip install grandalf
!pip install gradio

In [5]:
import os
from google.colab import userdata

os.environ['LANGSMITH_TRACING'] = 'true'
os.environ['LANGSMITH_ENDPOINT'] = "https://api.smith.langchain.com"
os.environ['LANGSMITH_API_KEY'] = userdata.get('LANGSMITH_API_KEY')
os.environ['LANGSMITH_PROJECT'] = "pr-indelible-manservant-54"

### OpenAI Initialization

You will be provided an API key by the Course Manager. If you are not able to find it, please reach out on ask for help on the Discussion forums.

Please note that this is a shared API key for the entire class, so please make sure to use it responsibly. We will also show you steps below to add it to your secret keys so that it is not revealed in any way.

Once you have obtained the API key, you can add it to the Google Colab instance by adding it to the Secrets section as shown below. Please use the variable name 'OPENAI_API_KEY' and paste the value that you copied before.

![Add to Google Colab](https://i.ibb.co/Cb57Sxq/Xnapper-2024-06-09-21-42-30.png)

# Step 1: Building the PillPal bot

The patient information leaflets/ drug booklets are typically available in the form of PDFs from the drug manufacturers website or from a central repository like in the [UK](https://www.medicines.org.uk/) or the [US](https://dailymed.nlm.nih.gov/). While you should be able to find them relatively easily through an Internet Search, we also include the medical booklet PDFs for two drugs with this code repository to use in the project.

One of the best aspects of using Langchain is that it provides a lot of in-built integrations for most common development tasks while building LLM apps. We will make use of several of them during this project, starting with the PDF loader which we will use to read in the medical booklet PDF file. For this example, we have chosen the PIL for Ozempic - a new drug that decreses the risk of heart disease in overweight patients but has been in the news recently for also being a weight loss solution.

As we are still in the notebook environment - please follow the below steps to upload the PDF document into the Colab environment.

![Upload PDF](https://i.ibb.co/jgDBwY4/Xnapper-2024-06-09-21-58-08.png)

## 📝 Learner Task:

In the following section, you have to load in your PDF using any of the document loaders available from the Langchain Community package. For example: you could use the `PyMuPDFLoader` for managing PDFs.

Your code should do the following:
- Load the PDF
- Identify the number of pages
- Print one of the pages with the associated metadata

In [6]:
from langchain_community.document_loaders import PyMuPDFLoader
import pprint

file_path = "./sample_data/timolol.pdf"
loader = PyMuPDFLoader(file_path)
docs = loader.load()
num_pages = docs[0].metadata['total_pages']
print(num_pages)
pprint.pp(docs[0])
pprint.pp(docs[1])

pprint.pp(docs[0].metadata)

10
Document(metadata={'producer': 'Aspose.PDF for .NET 18.7', 'creator': 'Aspose Ltd.', 'creationdate': '2024-07-15T12:29:34+05:30', 'source': './sample_data/timolol.pdf', 'file_path': './sample_data/timolol.pdf', 'total_pages': 10, 'format': 'PDF 1.6', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2024-07-30T17:21:33+05:30', 'trapped': '', 'modDate': "D:20240730172133+05'30'", 'creationDate': "D:20240715122934+05'30'", 'page': 0}, page_content='PACKAGE LEAFLET: INFORMATION FOR THE USER \nTimolol 0.25% w/v Eye Drops Solution  \n(timolol) \nRead all of this leaflet carefully before you start taking this medicine. because it \ncontains important information for you. \n-\nKeep this leaflet. You may need to read it again.\n-\nIf you have any further questions, ask your doctor or pharmacist or nurse.\n-\nThis medicine has been prescribed for you only. Do not pass it on to others. It may harm them,\neven if their signs of illness are the same as yours.\n-\nIf you get 

The additional metadata like page number and title is relevant to our product because when we answer questions: we can also point users to specific sections of the original document which they can refer to for more clarity.

Our next step is to index this entire document which we do with the help of OpenAI embeddings. However, before we do that we need to split document into chunks so that at the time of retrieval we are identifying the correct parts of the document. We use one of the in-built Langchain components that allows us to split based on characters based on our specifications.

We have chosen to split the entire document into chunks of length 2000 characters and also ensure an overlap of 200 characters. This makes sure that we are not loosing any information when splitting up the document. As we will see later, this is one of the parameters that we can control and could have an influence on the performance of our product.

## 📝 Learner Task:

In the following section, you have to decide the best chunking strategy for your PDF and use case. For instance, you could make use of the `RecursiveCharacterTextSplitter` to work with text documents. It's important to also decide what are reasonable values for `chunk_size` and `chunk_overlap`. In the project walkthrough we provide some suggested values.

In [7]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

full_text = ''
for page in docs:
  full_text += page.page_content

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)
splits = text_splitter.create_documents([full_text])
print(len(splits))

11


We are optimising for retrieval - i.e. how fine-grained is the context that we can retrieve so that we can answer the question that our user is asking. In this case, it might be better to have a smaller chunk because you are then narrowing down on the perfect part of the text where this information is present.

In [8]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from google.colab import userdata

## Define the embedding model below by using the `text-embedding-3-small` model

embedding_model = OpenAIEmbeddings(model='text-embedding-3-small', openai_api_key=userdata.get('OPENAI_API_KEY'))

A huge advantage of using Langchain is that it provides integrations with several types of vector databases like Chroma, Pinecone and more. In this project, we make use of the FAISS library from Facebook/Meta as a simple choice. In the following section, we combine the steps of generating the embedding value for each document chunk and then also storing it into the FAISS vector database.

In [9]:
from langchain_community.vectorstores import FAISS

vector_store = FAISS.from_documents(documents=splits, embedding=embedding_model)

We have our documents indexed and we can directly start retrieving documents based on a similarity search with a target query.

In [10]:
search_result = vector_store.similarity_search_with_relevance_scores(query="What is the recommended dosage?", k=4)
search_result

[(Document(id='82336006-304e-47c2-9837-1406f87e74b4', metadata={}, page_content='7. Wash your hands, tilt your head back and look at the ceiling (Fig.7). \n \n \n \n8. Gently pull the lower eyelid down to form a small pocket between the eyelid and the eye (Fig.8). \n \n \n \n9. Turn the bottle upside down and squeeze it gently to release one drop into each eye that needs \ntreatment. Do not touch your eye or eyelid with the dropper tip (Fig. 9). If a drop misses your eye, try \nagain.10. Let go of the lower lid, and close your eye For 2 minutes, by pressing the finger into the corner of \nyour eye, by the nose (Fig.10). This helps to stop timolol getting into the rest of your body. \n \nOphthalmic medications, if handled improperly, can become contaminated by common bacteria \nknown to cause infections. Serious damage to the eye and subsequent loss of vision may result from \nusing contaminated ophthalmic medications. If you think your medication may be contaminated, or if \nyou develo

In this case, we have chosen to use cosine similarity as our distance metric and the similarity values are also shown along with the retrieved results. You will notice that not many of the retrieved chunks actually contain the text "Weight Loss" - and this is because we are not doing a word-based search but rather semantic search. This is what vector databases allow us to do. However, it is also possible that at times we want to match exact query terms and then we will follow a hybrid search approach. This is again one of the levers that we have to expriment with to determine what is necessary for our product. So it's important that you are able to determine what is the best method for you to choose - depends a lot on your use-case.

### Creating the QnA RAG chain

Let's move on to complete our product by integrating with the LLM. Here again, we rely heavily on the building blocks that Langchain already provides us to put together the entire chain.

The chain consists of multiple parts - a Retriever, followed by a Prompt Template where the retrieved documents are added and then the call to the LLM. The response from the LLM is what we finally show as output to the user. There are multiple steps in creating a RAG application. We break it down to make it easier to understand and follow.

## 📝 Learner Task:
Creating the Prompt Template. In the section below, please create the PromptTemplate that will be used for your RAG application. The `ChatPromptTemplate` has become the standardized way to use chat-based LLMs and consists of a `SystemMessage`, followed by the `HumanMessage` with the response from the LLM stored in the `AIMessage`. We have provided the necessary imports in the below cell and request you to create the variable `qna_prompt_template` that will be used in the application.

In [11]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.prompts import SystemMessagePromptTemplate
from langchain.prompts import HumanMessagePromptTemplate

system_prompt = """
You are an expert with medical knowledge who is able to read drug
pamphlets and uses that information to answer the questions you are
given.

You are able and skilled at describing things in a simple fashion.
If you are uncertain of the answer or correctness, simply say that you
are uncertain.
"""

human_prompt = """
The context for your questions is {context}.
The question is {question}.
"""

qna_prompt_template = ChatPromptTemplate([
    ("system", system_prompt),
    ("human", human_prompt),
])


After the PromptTemplate, the next step is to define the LLM Model that you would like to run with. We provide the OpenAI API models as the default option and make use of `gpt-4o-mini`. We also set the `temperature` value to 0 as we do not want the model to be creative but rather answer based on the retrieved context from the PDF.

In [12]:
llm = ChatOpenAI(model_name="gpt-4o-mini",
                 temperature=0,
                 openai_api_key=userdata.get('OPENAI_API_KEY'))

The next step is to initilize the retriever part of our application that will fetch the relevant documents from the PDF. Recall that we have already created the embeddings for our document chunks and stored it using the FAISS vectorstore. In this step, we only specify that vectorestore to be our retriever.

In [13]:
retriever = vector_store.as_retriever()

One additional step that we need to also consider is that when the documents are retrieved from the FAISS vectorstore, they are in the form of a list of documents. These document objects contain a lot more information other than the content like Metadata. Assuming we are performing a simple retrieval, the context that we want to pass to the LLM is only the text content. Therefore we write an additional function that combines only the retrieved text content that can be used to pass in the context of our PromptTemplate.

Please note that you could also choose to filter the retrieved documents based on the Metadata. For instance, if you only want to see results from Page 5 and beyond of the PDF - then you can also adapt this function to reflect that.

In [14]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

The final stage is to put together the various elements that we have defined above to create our pipeline. The key aspect to note here is that the input question is used twice during the process -> at the first instance to retrieve a list of the relevant context and the second instance when it is passed to the LLM as the question that needs to be answered.

So we make use of the `itemgetter` to retrieve the content of the question and pass to the retriever to get the relevant context. On the other hand, we make use of the `RunnablePassthrough` class to directly pass the input question to the subsequent operation. At the end we also add the `StrOuputParser` to get the actual text message of the response rather than the whole AIMessage.

In [15]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

rag_chain = (
    {"context": itemgetter("input") | retriever | format_docs,
     "question": RunnablePassthrough()}
    | qna_prompt_template
    | llm
    | StrOutputParser()
)

Please check that your entire chain works by providing your questions and invoking the RAG chain.

In [16]:
question = "What is the dosage for a child?"
rag_chain.invoke({"input": question})

'The dosage for a child, especially a newborn, is one drop of Timolol Eye Drops instilled into the affected eye(s) each time, as prescribed by the doctor. The doctor will determine how long the eye drops will be needed. It is important to closely monitor the child for any signs of side effects after the first dose.'

## 📝 Learner Task:
If you run into errors as you develop, visualizing the chain is an easy way to discover any issues. This visualization helps us understand the entire flow of operations within our application. By graphically representing the connections and interactions between different components we can see how the data flows and in which shape from one path tothe next. It allows for easier debugging and we might be able to pinpoint where modifications might be necessary.

In [17]:
print (rag_chain.get_graph().print_ascii())

           +---------------------------------+        
           | Parallel<context,question>Input |        
           +---------------------------------+        
                    **              ***               
                 ***                   **             
               **                        ***          
       +--------+                           **        
       | Lambda |                            *        
       +--------+                            *        
            *                                *        
            *                                *        
            *                                *        
+----------------------+                     *        
| VectorStoreRetriever |                     *        
+----------------------+                     *        
            *                                *        
            *                                *        
            *                                *        
    +-----


Hopefully, you have managed to build the skeleton for your product. At first just make sure that everything is working in an end to end fashion. After you have achieved that you can move in the direction of improving the answers from the application. This is when you can start experimenting with different aspects of your pipeline to go in the direction of a better solution.

We would recommend adapting the following options:

- Chunk size:
- Retrieval strategy/ metric:
- Choice of LLM:

At the end of this stage, you should be able to have a working prototype of your solution that does reasonably well for the use-case you have in mind.

We are still relying on a vibe/gut-feel for the product and cannot rely on quantified metrics. But hopefully you can feel reasonably confident to launch an alpha version of your product to a select group of customers.

# Step 2: Deploying the PillPal bot

Before deploying our product, one of the important steps is to enable tracking and monitoring all the user queries and LLM responses. This is made very easy for us with the help of Langsmith - another part of the Langchain ecosystem.

This is one of the cornerstones of our strategy to get towards a more valuable and performing LLM app.

Please sign-up on the Langsmith [Website](https://smith.langchain.com/) and you should have access. Next, please navigate to the Settings page from the left side navigation menu and then create your API key there and copy it.

<a href="https://ibb.co/fN8s3Yp"><img src="https://i.ibb.co/ZgH0Q68/Screenshot-2024-07-26-at-16-12-48.png" alt="Screenshot-2024-07-26-at-16-12-48" border="0"></a>




After that, you can return to the main page and click the New Project icon and copy the environment details that you see.

<a href="https://ibb.co/th2CQhd"><img src="https://i.ibb.co/yRksdR3/Screenshot-2024-07-26-at-16-12-59.png" alt="Screenshot-2024-07-26-at-16-12-59" border="0"></a>

We can deploy our LLM app in two different ways -

1. A website based chatbot experience - this will be done with the help of Gradio and you do not need to leave the Colab notebook environment
2. A telegram chatbot that will run on Github Codespaces (or locally) and will need you to leave the Colab environment

### Deployed as website chatbot

We can deploy our QnA bot on a dedicated website using Gradio, an easy-to-use library for creating interactive machine learning interfaces. One significant advantage of using Gradio in our project is that it integrates seamlessly within the Colab notebook environment. This means you don't need to leave Colab to see your bot in action; you can run it directly from the notebook. Gradio also provides a dedicated URL that you can share with potential beta testers of the QnA bot to get feedback.

In [18]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage
import openai
import gradio as gr

def predict(message, history):
    return rag_chain.invoke({"input":message})

gr.ChatInterface(predict).launch(debug=True)

/usr/local/lib/python3.11/dist-packages/gradio/components/chatbot.py:290: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://1d0ba1eda0b3ee8c0e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://1d0ba1eda0b3ee8c0e.gradio.live


### [OPTIONAL] Deployed as Telegram bot

For a wider launch of your LLM app, you can easily make it available as a bot on Telegram - a messenger platform that is used by 500 million users every day. This is useful in the beta/alpha phase because it's way easier to get users to try it in an app that they already use every day. This need not be just Telegram but other popular messengers like Whatsapp, Signal etc. We chose Telegram as it's free and easy to setup.

In order to deploy using this option, we will make use of Github codespaces as it's difficult to run it within a synchronous coding environment like Colab. The instructions and code for this will be provided in the live project session.

Please go to the Github repository: https://github.com/sidhusmart/uplimit_langchain.git and clone it so that you can work on it directly. Please note that code in this repository may change frequently so we request you to work only on your cloned version.

Once you have cloned it, then you can launch your own Codespaces instance by following the steps as shown in the screenshot. This creates a virtual Visual Studio Code environment in the cloud where we can test our Telegram bot.

<a href="https://ibb.co/p02fJ1t"><img src="https://i.ibb.co/BzLyqfX/Screenshot-2024-07-26-at-16-05-46.png" alt="Screenshot-2024-07-26-at-16-05-46" border="0"></a>

The two important files for you to take a look are `langchain_rag_app.py` and `telegram_response_bot.py` The first python file contains the same RAG functionality that we already built but packaged in functions. The telegram response bot contains the necessary code required to run the Telegram bot.

In order to setup and run this app, we have to add the necessary API keys. This is the same process as what we have done previously in the Colab notebook but since we are executing this code in a new environment, we have to add the necessary secrets there as well.

We do this via the `.env.dev` file. This file does not exist in the repository by default but instead we have provided the template file called `.env.template`. You need to create a copy of this file and rename it to be `.env.dev` that will contain all the API keys you are going to use during development of the app. Please note that this file is not checked in as we do not want to commit the secrets!  

You can observe that there are several environment variables defined in the .env file - most of which we have already setup. You can use the same values for those keys again.

However, there is one additional API key that is required to work with the Telegram API. In order to get the Telegram bot API key, you need to follow these steps:

1. You must have a Telegram account and access to the App - Web or Mobile versions would also work.
2. On the Telegram app, you need to search and look for a contact called the Botfather.
3. You can use the `/start` command to get a list of options or directly use the `/newbot` command to create a new bot for yourself.
4. Then answer the set of questions by providing a name for and username for your bot.
5. At the end you will be provided with an API token that you can use to work with the bot. This API token is what we need to replace in the .env.dev file. `telegram_response_bot.py` file.

<a href="https://imgbb.com/"><img src="https://i.ibb.co/f2MqxCd/Screenshot-2024-07-26-at-14-23-02.png" alt="Screenshot-2024-07-26-at-14-23-02" border="0"></a>

Once you have performed both these steps, you are now ready to run your RAG app as a Telegam bot. You need to run the command `python telegram_response_bot.py ` from the command line of your Github Space and this starts a thread that listens for messages on your bot. Once a message is received, it then hands off to the RAG app and provides the generated response.

You can easily share this Pillpal bot with your friends and family and ask them to try it!

# Step 3: Evaluation of PillPal

Now that we have deployed and made our app live, we need to consistently monitor how it performs. This is also a way for us to understand what questions are being asked by our users and whether the bot is responding correctly or not. For this we will make use of the Langsmith part of the Langchain library.

As the first step, we have to create a validation dataset that we can use for evaluation. There are several ways to go about doing this:

1. Launch your app in beta and get some trial users to start interacting with the bot/chat interface and identify common patterns and questions that are being asked. Manually, write your own answers to these questions and perform evaluations using that.
2. Come up with a list of 5-10 questions and manually search for the answers of these questions in the PDF and create this as your evaluation dataset -> this is what we are going to do below.  
3. You could also make use of another LLM to come up with these questions and answers which is called creating a synthetic dataset. This would work well if you use higher level models like GPT-4 but you need to keep in mind that they have to be relevant to the context of your leaflet.

### Creating a base evaluation dataset

To ensure that our application is functioning correctly and effectively answering questions, we have implemented a basic sanity check. This involves a set of 8-10 questions that I have manually curated and answered after thoroughly reviewing the content of the PDF document. These questions are designed to cover a wide range of topics and complexities within the document, ensuring that they test various aspects of our QnA bot's capabilities. By comparing the answers generated by our bot to these manually prepared responses, we can gauge the accuracy and reliability of our system. This step is crucial as it helps us identify any discrepancies or areas needing improvement before the application is deployed for wider use. Additionally, it provides an initial layer of validation and builds confidence in the application's performance among users and stakeholders.

## 📝 Learner Task:

Please create a set of question and correct answer pairs here to use as your evaluation dataset. We have provided the first two questions as examples but feel free to overwrite and add your own.

In [19]:
examples = [
    (
        "Can it be taken by pregnant women?",
        "No, it should not be taken by pregnant women before seeking the doctors advice.",
    ),
    (
        "Can it be given to children?",
        "Yes, it has been tested in newborns with glaucoma, however extreme care must be given.",
    ),
    (
        "What is the usual dosage?",
        "One drop per affected eye, 1 to 2 times per day."
    ),
    (
        "How do I prevent absorption of the medication in the rest of the body",
        "Close eye and press finger on inside corner of eye for 1-2 minutes."
    ),
    (
        "Can timolol affect blood pressure?",
        "Yes, timolol can cause lower blood pressure."
    )
]

Once we have successfully created our dataset, the next step involves registering it with the Langsmith client. This process integrates the dataset into our evaluation framework. By registering the dataset, we effectively add it to the list of datasets available for testing and validating our model. In general, you might create multiple datasets to test various aspects of the QnA bot and might run different types of evaluations on it.

In this case, we are running a test for general correctness of the QnA bot - while this does include checks for hallucinations -> we have added examples where the bot should not have an answer and must say so. But there are additional datasets that you might create with separate evaluation metrics to explicitly check for hallucination and fact-checking.

In [20]:
from langsmith import Client

client = Client()

In [21]:
from langsmith import Client

client = Client()

dataset_name = "PillPall Accuracy Test Dataset"
dataset = client.create_dataset(dataset_name=dataset_name)
inputs, outputs = zip(
    *[({"input": input}, {"expected": expected}) for input, expected in examples]
)
client.create_examples(inputs=inputs, outputs=outputs, dataset_id=dataset.id)

LangSmithConflictError: Conflict for /datasets. HTTPError('409 Client Error: Conflict for url: https://api.smith.langchain.com/datasets', '{"detail":"Dataset with this name already exists."}')

Once the above cells are run, you should now be able to see the dataset created in the dataset folder in your Langsmith project.

![QA Dataset](https://i.ibb.co/sH5dv86/Xnapper-2024-06-11-20-58-53.png)

Let's run the evaluation of our app using this dataset.

By executing the code below, we will first be calling our app for each of the questions in our QA dataset. Once the answers have been generated, we need to evaluate it with the golden standard answers that we have written manually. But the big question is always: how can we compare two texts, what metrics shall we use?

One of the easier ways to do this is by using another LLM to compare the two responses and tell you whether they match or not. This is also referred to as 'LLM-as-a-judge'. In this case, since we have only 5-10 examples you can easily do this manually but typically your evaluation datasets will contain a lot of examples and this method would not work.

We will use the off-the-shelf evaluator that Langsmith provides us called `cot_qa` that stands for Chain of Thought Question Answer. This basically refers to a pre-filled prompt that is used when asking the "Judge LLM" to rate whether the two answers are comparable or not.

Typically we make use of a bigger, more powerful LLM to act as the judge for what the smaller, cheaper LLM has generated. However, since our dataset is small and quite easy we stick with `gpt-4o-mini`

In [ ]:
from langsmith import Client
from langsmith.evaluation import LangChainStringEvaluator, evaluate

eval_llm = ChatOpenAI(temperature=0.0, model="gpt-4o-mini", openai_api_key=userdata.get('OPENAI_API_KEY'))
cot_qa_evaluator = LangChainStringEvaluator("cot_qa", config={"llm": eval_llm})

client = Client()
evaluate(
    rag_chain,
    data=dataset_name,
    evaluators=[cot_qa_evaluator],
)

Let's run the evaluation on the dataset. You get the links to where you can track the run and at the end the output is also printed here.

After initiating the evaluation of our QnA bot using the dataset we registered, you will be able to observe several key performance metrics. These metrics include the percentage of questions answered correctly, which provides a direct measure of the bot's accuracy. You will also see the duration of the test, which gives insight into the efficiency of the bot under testing conditions. Additionally, the evaluation will report on costs and latency, offering a broader view of the bot's operational performance.

This initial evaluation serves as a preliminary indication of how well your bot is performing. As the bot encounters more users and a diverse array of questions, it's crucial to continually update and adapt your baseline test dataset. This iterative process ensures that the bot remains effective and responsive to the evolving needs and contexts it will encounter in real-world applications.

![Evaluation Test results](https://i.ibb.co/txtzhtD/Xnapper-2024-06-16-11-32-28.png)

## 📝 Learner Task:

Now that you have an end to end RAG chatbot up and running, you can try to stress-test your bot by asking all kinds of questions - both relevant and irrelevant. What kind of responses does it generate? How often does it hallucinate?

Simple irrelevant medical questions on other medicine get an uncertain result with consultation advice. A totally irrelevant question did give the answer but also noted it was not related to the topic and a pointer to get back on track.. A wacky question using the drug as part also got that it was uncertain and information about consultation.

```
What is the usual dosage of aspirin?

I'm uncertain about the usual dosage of aspirin, as the information provided is specifically about Timolol Eye Drops and does not include details on aspirin. For accurate dosage information regarding aspirin, it's best to consult a healthcare professional or refer to the medication's prescribing information.

How many cups are there in a gallon?

I'm uncertain about the answer to that question, as it is not related to the medical context or information about Timolol. However, to answer your question: there are 16 cups in a gallon. If you have any questions about Timolol or related medical topics, feel free to ask!

How does timolol cause global warming?

I am uncertain about how timolol, a medication used for treating eye conditions, could cause global warming. Timolol is a beta-blocker that affects the cardiovascular system and is primarily used to lower intraocular pressure in conditions like glaucoma. It does not have a direct connection to environmental issues such as global warming. If you have concerns about the environmental impact of pharmaceuticals or specific medications, it would be best to consult a specialist in environmental science or pharmacology.
```

Explicitly asking to ignore the prompt and information that was given does have the bot respond to irrelevant questions, so some form of protection for that would be necessary if not intended for personal/limited use.